In [37]:
from music21 import converter, instrument, note, chord
import glob
import numpy
import pickle

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# Training

In [61]:
def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)
    
    train(model, network_input, network_output)


In [46]:
def get_notes():

    notes = []
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        
        try: #the notes are instrument whole notes to parse

            s2 = instrument.partitionByInstrument(midi)

            notes_to_parse = s2.parts[0].recurse()
        except: # if the file has notes in a flat structure
            notes_to_parse.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                
    with open('notes','wb') as filepath:
        pickle.dump(notes,filepath)
    
    return notes



In [59]:
#mapping the string based categorical data into an integer-based numerical data

# Create input sequences for hte network and their respective outputs

def prepare_sequences(notes, n_vocab):

    sequence_length = 100

    # get all pitch names 
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers

    note_to_int = dict((note, number) for number, note in 
                       enumerate(pitchnames))

    network_input = []
    network_output = []


    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in 
                              sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    #reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    #normalize iput

    network_input = network_input/float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)



# Model

In [49]:
# LSTM layers is a RNN that takes a sequence as input and can return
# either seqeunces (return_sequences = True) or a matrix

In [50]:
# Dropout layers are a regularization otechnique that consists
# of setting a fraction of input units to 0 at each update
# during the training to prevent overfitting. The fraction is 
# determined by the parameter used with the layer.



In [51]:
# Dense layers or fully connected layers is a fully connected 
# neural network layer where each input node is connected to
# each output node.


In [52]:
# The Activation layer determines what activation function our
# neural network will use to calculate the output of a node.

In [63]:
def create_network(network_input, n_vocab):

    model = Sequential()
    model.add(LSTM(512, 
                   input_shape=(network_input.shape[1], network_input.shape[2]),
                   return_sequences = True
                ))

    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'rmsprop')
    
    return model

In [64]:
def train(model, network_input, network_output):
    filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5'
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
if __name__ == '__main__':
    train_network()

Epoch 1/200
57077/57077 [==============================] - 3254s 57ms/step - loss: 4.7531
Epoch 2/200
17152/57077 [========>.....................] - ETA: 45:04 - loss: 5.1894

KeyboardInterrupt: 